<a href="https://colab.research.google.com/github/AlekseyZykov/Python/blob/main/%D0%9F%D1%80%D0%BE%D0%B4%D1%83%D0%BA%D1%82%D0%BE%D0%B2%D1%8B%D0%B5_%D0%BC%D0%B5%D1%82%D1%80%D0%B8%D0%BA%D0%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Продуктовые метрики

**Важно!**

В следующей ячейке скачиваются нужные файлы

In [ ]:
!wget https://gist.github.com/Vs8th/739269a03f2f4a7396d04d6739da3771/raw/registrations.csv

!wget https://gist.github.com/Vs8th/aacb80595d1d6aaa2e31eb735f8bc644/raw/entries.csv

!wget https://gist.github.com/Vs8th/0e827e9a608117345dd6585ab81e8c86/raw/metrics.txt

Файлы выглядят следующим образом

In [ ]:
import pandas as pd
reg = pd.read_csv('registrations.csv', sep=';')
reg.head()

In [ ]:
ent = pd.read_csv('entries.csv', sep=';')
ent.head()

In [ ]:
# Подготовка таблиц к работе

with open('registrations.csv') as f:
    reg_rows = f.readlines()[1:]  # пропускаем строку заголовка

# собираем первый файл в список словарей
registrations = []
for row in reg_rows:
    user_id = int(row.split(";")[0])
    registration_date = row.split(";")[1].strip()
    registrations.append({"user_id": user_id, "registration_date": registration_date})

# то же самое с вторым файлом
with open('entries.csv') as f:
    entries_rows = f.readlines()[1:]

entries = []
for row in entries_rows:
    user_id = int(row.split(";")[0])
    entry_date = row.split(";")[1].strip()
    entries.append({"user_id": user_id, "entry_date": entry_date})
    # Убераем записи о входах, где один юзер зашел дважды за день

users_on_day = set()

filtered_entries = []

for entry in entries:
    user_id = entry['user_id']
    entry_date = entry['entry_date']

    if (user_id, entry_date) in users_on_day:
        continue
    else:
        users_on_day.add((user_id, entry_date))
        filtered_entries.append(entry)

#### Retention 15 дня для пользователей, зарегистрированных в январе


In [ ]:
from datetime import datetime

# фильтруем январские регистрации
january_registrations = []
for registration in registrations:
    if registration["registration_date"].split("-")[1] == "01":
        january_registrations.append(registration)

# объединяем по user_id январские регистрации и входы
merged = []
for registration in january_registrations:
    for entry in filtered_entries:
        if entry["user_id"] == registration["user_id"]:
            merged.append({"user_id": entry["user_id"], "registration_date": registration["registration_date"], "entry_date": entry["entry_date"]})

# считаем количество дней с регистрации
days_since_registration = []
for m in merged:
    registration_date = datetime.strptime(m["registration_date"], "%Y-%m-%d")
    entry_date = datetime.strptime(m["entry_date"], "%Y-%m-%d")
    days = (entry_date - registration_date).days
    days_since_registration.append({"user_id": m["user_id"], "days_since_registration": days})

# фильтруем тех, кто вернулся в 15 день
returned_in_15_days = set()
for d in days_since_registration:
    if d["days_since_registration"] == 15:
        returned_in_15_days.add(d['user_id'])

# считаем retention
retention_15_day = round(len(returned_in_15_days) / len(january_registrations) * 100, 5)
round(retention_15_day, 5)

#### Rolling-retention 30 дня для пользователей из той же когорты


In [ ]:
# создаем множество для хранения юзеров зашедших в 30 день или после
rolling_retention_rates = set()

for i in days_since_registration:
    if i['days_since_registration'] >= 30:
        rolling_retention_rates.add(i['user_id'])

# считаем rolling retention
rolling_retention = round(len(rolling_retention_rates)/len(january_registrations) * 100, 5)
round(rolling_retention, 5)

#### Lifetime по всем пользователям, посчитанный как интеграл от n-day retention


In [ ]:
# объединяем регистрации и отфильтрованные входы по user_id
mergedallusers = []
for registration in registrations:
    for entry in filtered_entries:
        if entry["user_id"] == registration["user_id"]:
            mergedallusers.append({"user_id": entry["user_id"], "registration_date": registration["registration_date"], "entry_date": entry["entry_date"]})

# считаем количество дней с регистрации для каждого входа каждого пользователя
days_since_registration_all = []
for m in mergedallusers:
    from datetime import datetime
    registration_date = datetime.strptime(m["registration_date"], "%Y-%m-%d")
    entry_date = datetime.strptime(m["entry_date"], "%Y-%m-%d")
    days = (entry_date - registration_date).days
    days_since_registration_all.append({"user_id": m["user_id"], "days_since_registration": days})

counts = {}
for entry in days_since_registration_all:
    days_since_registration = entry['days_since_registration']
    if days_since_registration not in counts:
        counts[days_since_registration] = 0
    counts[days_since_registration] += 1
each_day_ret = dict(sorted(counts.items()))

day_zero_count = each_day_ret[0]
lifetime = 0

for day, count in each_day_ret.items():
    lifetime += (count / day_zero_count)

lifetime = round(lifetime, 5)

#### Churn rate 29 дня, посчитанный по всем пользователям


In [ ]:
cr_29 = set()
# churn rate n дня - юзеры которые на n день отвалились от продукта.
# Проверяем были ли у них заходы после n дня,
# если были, то добавляем в сет.
for i in days_since_registration_all:
    if i['days_since_registration'] >= 29:
        cr_29.add(i['user_id'])

churn_29 = (len(registrations)-len(cr_29))/len(registrations)
churn_29

#### Mau / wau / dau (разные способы расчета - за последний месяц, усредненные за весь период и т.д.)

##### Mau, wau, dau за последний месяц/неделю/день записей


**Примечание:** последний месяц записей - декабрь. Поэтому `mau` рассчитываем для декабря (2021 года), для `wau` берем последнюю неделю - с 25 по 31 декабря, и для `dau` соответственно последний день - 31 декабря.

In [ ]:
# получаем входы за декабрь
december_user_entries = []
for i in filtered_entries:
    if i['entry_date'][:4] == "2021" and i['entry_date'][5:7] == '12':
        december_user_entries.append(i)

In [ ]:
# рассчитываем mau

december_2022_mau = set()
for i in december_user_entries:
    december_2022_mau.add(i['user_id'])

dec_mau = len(december_2022_mau)

In [ ]:
# тоже самое для wau

december_wau = set()
for i in december_user_entries:
    if i['entry_date'][-2:] in ('25', '26', '27', '28', '29', '30', '31'):
        december_wau.add(i['user_id'])

dec_wau = len(december_wau)

In [ ]:
# dau

december_dau = set()
for i in december_user_entries:
    if i['entry_date'][-2:] == '31':
        december_dau.add(i['user_id'])

dec_dau = len(december_dau)

##### Mau/wau/dau усредненные

In [ ]:
from collections import defaultdict

# Создаем defaultdict для хранения уникальных юзеров каждог месяца
counts_avg = defaultdict(set)

# Проходим по входам
for line in entries_rows:
    # Выделим user_id и дату входа
    user_id, entry_date = line.strip().split(';')

    # Вытаскиваем год и месяц
    year, month = entry_date.split('-')[:2]

    # Используем год и месяц в качестве ключа
    key = f"{year}-{month}"
    counts_avg[key].add(user_id)

# Конвертируем множество пользователей в количество уникальных пользователей
count_of_unique_user_ids = {}
for key, value in counts_avg.items():
    count_of_unique_user_ids[key] = len(value)

In [ ]:
# рассчитываем усредненный mau

sum = 0
for key, value in count_of_unique_user_ids.items():
    sum += value

avg_mau = round(sum/len(count_of_unique_user_ids), 5)

In [ ]:
from collections import defaultdict

from datetime import datetime

counts = defaultdict(set)

for line in entries_rows:
    user_id, entry_date = line.strip().split(';')

    # Преобразуем строку с датой в объект datetime
    date_obj = datetime.strptime(entry_date, '%Y-%m-%d')

    # Получаем номер недели в году и год с помощью метода isocalendar()
    year, week_num, _ = date_obj.isocalendar()

    # Формируем ключ в формате "ГОД-НОМЕР_НЕДЕЛИ"
    key = f"{year}-{week_num}"
    counts[key].add(user_id)

# Конвертируем множество пользователей в количество уникальных пользователей
count_of_unique_user_ids = {}
for key, value in counts.items():
    count_of_unique_user_ids[key] = len(value)


In [ ]:
# считаем усредненный wau

sum = 0
for key, value in count_of_unique_user_ids.items():
    sum += value

avg_wau = round(sum/len(count_of_unique_user_ids), 5)

In [ ]:
# Все то же самое для каждого дня
counts = defaultdict(set)

for line in entries_rows:
    user_id, entry_date = line.strip().split(';')

    # В качестве ключа используем дату входа
    counts[entry_date].add(user_id)

# Конвертируем множество пользователей в количество уникальных пользователей
count_of_unique_user_ids = {}
for key, value in counts.items():
    count_of_unique_user_ids[key] = len(value)

sum = 0
for key, value in count_of_unique_user_ids.items():
    sum += value

avg_dau = round(sum/len(count_of_unique_user_ids), 5)